# Subfields

In this tutorial, we demonstrate several operations that can be used for extracting parts of the field. More precisely, we are going to show how to:

1. Get an individual component of a vector field
2. Extract smaller region of the field
    1. without resampling
    2. with resampling
    
Let us first create a vector field to work with:

In [2]:
import discretisedfield as df

p1 = (0, 0, 0)
p2 = (100, 100, 100)
cell = (2, 2, 2)
region = df.Region(p1=p1, p2=p2)
mesh = df.Mesh(region=region, cell=cell)
field = df.Field(mesh, dim=3, value=(-1, 2, 6))

## Getting a single component

From a vector field we can get its $x$, $y$, or $z$ component by simply asking for the right property. For instance:

In [3]:
field.x

Field(mesh=Mesh(region=Region(p1=(0, 0, 0), p2=(100, 100, 100)), n=(50, 50, 50), bc='', subregions={}), dim=1)

In [4]:
field.y

Field(mesh=Mesh(region=Region(p1=(0, 0, 0), p2=(100, 100, 100)), n=(50, 50, 50), bc='', subregions={}), dim=1)

In [5]:
field.z

Field(mesh=Mesh(region=Region(p1=(0, 0, 0), p2=(100, 100, 100)), n=(50, 50, 50), bc='', subregions={}), dim=1)

They all result in a scalar field and from this point on, we can perform all usual operations on fields. For example,

In [7]:
df.integral(field.x * df.dV)

-1000000.0

## Extracting smaller region

Let us say we are not interested in the entire field but only in its smaller portion (only some discretisation cells) and want to analyse it. In that case, we have two options. Before we discuss them, let us first define what we mean by "aligned" meshes:

- Mesh A is aligned to mesh B if and only if all cell coordinates of mesh A are the coordinates of cells in mesh B.

There is `|` operator which checks that: